# TASK 1

In [1]:
# Install required packages (if not already installed)
# !pip install pykeen torch pandas scikit-learn

import pandas as pd
import torch
import numpy as np
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from sklearn.metrics.pairwise import euclidean_distances

# -----------------------------
# STEP 1: Load and Prepare Triples
# -----------------------------
# Load triples from TSV
tf = TriplesFactory.from_path("C.1_query.tsv", delimiter="\t")
# Split into train/test (80/20 split) (do we need a validation set?)
train_tf, test_tf = tf.split()

# -----------------------------
# STEP 2: Train TransE Model
# -----------------------------
result = pipeline(
    model='TransE',
    training=train_tf,
    testing=test_tf,
    model_kwargs=dict(embedding_dim=50),
    training_kwargs=dict(num_epochs=100),
)

# Access trained model and embedding lookup
model = result.model
entity_embeddings = model.entity_representations[0]
relation_embeddings = model.relation_representations[0]

using automatically assigned random_state=2140251120
No random seed is specified. Setting to 7279976.
No cuda devices were available. The model runs on CPU


Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Evaluating on cpu:   0%|          | 0.00/2.89k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 2.34s seconds


In [ ]:
# -----------------------------
# STEP 3: Reasoning with Embeddings
# -----------------------------
# Replace this with the actual paper URI from your KG
paper_uri = "<http://example.org/research_paper/4ab60d22-67d1-4683-8648-f1f2601d2ce0>"
cites_uri = "<http://example.org/research_paper/cites>"
written_by_uri = "<http://example.org/research_paper/written_by>"

# Get internal IDs for these entities/relations
entity_to_id = result.training.entity_to_id
relation_to_id = result.training.relation_to_id

paper_id = entity_to_id.get(paper_uri)
cites_id = relation_to_id.get(cites_uri)
written_by_id = relation_to_id.get(written_by_uri)

if paper_id is not None and cites_id is not None and written_by_id is not None:
    # Get embeddings for paper and relations
    paper_vec = entity_embeddings(torch.tensor([paper_id]))[0].detach().numpy()
    cites_vec = relation_embeddings(torch.tensor([cites_id]))[0].detach().numpy()
    written_by_vec = relation_embeddings(torch.tensor([written_by_id]))[0].detach().numpy()

    # Step 3a: Estimate embedding of most likely cited paper
    target_cited_paper_vec = paper_vec + cites_vec

    # Step 3b: Estimate embedding of its author
    hypothetical_author_vec = target_cited_paper_vec + written_by_vec

    # -----------------------------
    # STEP 4: Find Closest Author Entity
    # -----------------------------
    all_entity_ids = list(model.triples_factory.entity_to_id.values())
    all_entity_labels = list(model.triples_factory.entity_to_id.keys())
    all_embeddings = entity_embeddings(torch.tensor(all_entity_ids)).detach().numpy()

    distances = euclidean_distances([hypothetical_author_vec], all_embeddings)[0]
    closest_idx = np.argmin(distances)
    closest_entity = all_entity_labels[closest_idx]
    print(f"The most likely author entity is: {closest_entity}")

else:
    print("One or more URIs not found in the model. Check your KG identifiers.")
